In [ ]:
token = "DRFAgjossvsFMnlCm5pGu6x3iIMOiZrKVnKvbYOYax_K"



In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService.save_account(
  token=token, # Your token is confidential.
  # Do not share your token in public code.
  instance="cphase", # Optionally specify the instance to use.
  region="us-east", # Optionally set the region to prioritize. This is ignored if the instance is specified.
  set_as_default=True, # Optionally set these as your default credentials.
  overwrite=True
)

In [3]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit.library import QFT

from qiskit_ibm_runtime import QiskitRuntimeService

import os
os.environ['HTTP_PROXY'] = 'http://localhost:7891'
os.environ['HTTPS_PROXY'] = 'http://localhost:7891'

service = QiskitRuntimeService()
backend_fractional = service.backend("ibm_torino", use_fractional_gates=True)
backend_conventional = service.backend(
    "ibm_torino", use_fractional_gates=False
)
 
pm_fractional = generate_preset_pass_manager(
    optimization_level=3, backend=backend_fractional, scheduling_method="alap"
)
pm_conventional = generate_preset_pass_manager(
    optimization_level=3,
    backend=backend_conventional,
    scheduling_method="alap",
)

qft_circuit = QFT(6, do_swaps=False).decompose()
qft_circuit.measure_all() # 添加测量操作
 
qft_circuit_fractional = pm_fractional.run(qft_circuit)
qft_circuit_conventional = pm_conventional.run(qft_circuit)

/var/folders/bp/6bnz0bhn5g977qdfbg62ksqm0000gn/T/ipykernel_25472/825412819.py:25: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qft_circuit = QFT(6, do_swaps=False).decompose()


In [7]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler

def submit_job(qc: QuantumCircuit, backend, shots=4096):
    # 真机，noisy数据
    sampler = Sampler(backend)
    job = sampler.run([qc], shots=shots)
    return job.job_id()

# jobid_cx = submit_job(qft_circuit_conventional, backend_conventional)
# jobid_frac = submit_job(qft_circuit_fractional, backend_fractional)
jobid_cx = "d2a5kfrac8as73amh850"
jobid_frac = "d2a5num682lc73d6j6hg"

print(f">>> Job ID: {jobid_cx}")
print(f">>> Job ID: {jobid_frac}")


>>> Job ID: d2a5kfrac8as73amh850
>>> Job ID: d2a5num682lc73d6j6hg


In [13]:
import qiskit.quantum_info as qi

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler


def get_ideal_counts(qc: QuantumCircuit, shots=4096):
    # Aer 模拟器，不含噪声数据
    backend = AerSimulator()
    return backend.run(qc, shots=shots).result().get_counts()

def get_noisy_counts(jobid):
    # 真机，noisy数据
    job = service.job(jobid)
    job_result = job.result()
    result = job_result[0].data.meas.get_counts()
    return result


def get_program_fidelity(qc, jobid, shots=4096):
    ideal_counts = get_ideal_counts(qc, shots)
    print(f">>> Ideal counts: {ideal_counts}")
    noisy_counts = get_noisy_counts(jobid)
    print(f">>> Noisy counts: {noisy_counts}")
    return qi.hellinger_fidelity(ideal_counts, noisy_counts)


fid_cx = get_program_fidelity(qft_circuit_conventional, jobid_cx)
fid_frac = get_program_fidelity(qft_circuit_fractional, jobid_frac)

print(f">>> Fidelity CX: {fid_cx}")
print(f">>> Fidelity Fractional: {fid_frac}")


>>> Ideal counts: {'001010': 53, '010100': 71, '101001': 69, '000110': 58, '010011': 63, '011110': 57, '101110': 57, '100100': 66, '101010': 70, '001111': 83, '110101': 66, '110110': 64, '001110': 74, '100111': 76, '101100': 60, '000011': 67, '001100': 64, '101000': 50, '100010': 70, '100011': 65, '001000': 72, '111100': 60, '111010': 56, '001001': 60, '011011': 58, '001011': 63, '010110': 65, '010001': 57, '010111': 71, '001101': 57, '010000': 62, '100110': 60, '011000': 68, '011111': 66, '110000': 63, '011101': 69, '000000': 70, '110001': 66, '000101': 66, '010101': 68, '010010': 67, '011100': 78, '101111': 68, '100000': 58, '111110': 67, '000111': 69, '111111': 63, '101101': 64, '110011': 69, '000010': 71, '111000': 59, '100101': 60, '000001': 55, '011010': 52, '111101': 50, '111001': 75, '100001': 56, '110111': 62, '011001': 58, '110100': 72, '110010': 57, '101011': 53, '111011': 69, '000100': 64}
>>> Noisy counts: {'111101': 48, '111010': 78, '101001': 84, '000111': 66, '010101': 

In [ ]:
# from qiskit.primitives import Estimator

# estimator = Estimator(mode=backend_fractional)
# estimator.options.resilience_level = 1
# estimator.options.default_shots = 5000

# result = estimator.run([qft_circuit_fractional, qft_circuit_conventional], [1, 1])
# print(result)

# mapped_observables = [
#     observable.apply_layout(isa_circuit.layout) for observable in observables
# ]
 
# # One pub, with one circuit to run against five different observables.
# job = estimator.run([(isa_circuit, mapped_observables)])
 
# # Use the job ID to retrieve your job data later
# print(f">>> Job ID: {job.job_id()}")

global phase: 5.5223
          ┌────────────────┐   ┌────┐                                        »
q_2 -> 55 ┤ Delay(134[dt]) ├───┤ √X ├────────────────────────────────────────»
          ├───────────────┬┘   ├────┤                                        »
q_3 -> 64 ┤ Delay(25[dt]) ├────┤ √X ├────────────────────────────────────────»
          └┬──────────────┤    └────┘                         ┌───────────┐  »
q_4 -> 65 ─┤ Delay(8[dt]) ├────────────────────────■──────────┤ Rz(-3π/4) ├──»
           └─┬──────────┬─┘ ┌─────────┐┌─────────┐ │ZZ(π/4) ┌─┴───────────┴─┐»
q_5 -> 66 ───┤ Rz(-π/2) ├───┤ Rx(π/2) ├┤ Rz(π/2) ├─■────────┤ Delay(75[dt]) ├»
          ┌──┴──────────┴──┐└─────────┘└─────────┘          └───────────────┘»
q_1 -> 67 ┤ Delay(265[dt]) ├─────────────────────────────────────────────────»
          ├────────────────┤                                                 »
q_0 -> 74 ┤ Delay(400[dt]) ├─────────────────────────────────────────────────»
          └────────────────┘   